In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import strategy.trendlabeling as tlb
import numpy as np
import datetime

import getdata as gd
import afml.filters as flt 
from afml.util.volatility import getDailyVolatility
import features.bars as bars

1. Import Dataset

In [43]:
df = gd.get_yf_data(tickers= "SPY AAPL ALGM DNOW", 
                    period='7d',  #'1y', 
                    interval='1m')
df = df[df['Ticker'] == 'ALGM']

[*********************100%***********************]  4 of 4 completed


2. Form Dollar Bars

In [51]:
_df = df.reset_index()
_df = _df[['Open', 'High', 'Low', 'Adj Close', 'Date', 'Volume']]
_df.columns = ['close', 'high', 'low', 'open', 'time', 'vol']
_df.time = pd.to_datetime(_df.time, format="%Y-%m-%dT%H:%M:%S.%fZ") 
df_dict = _df.to_dict('records')

In [61]:
# dollar bar threshold = 
# 30 day moving average of volume and then dividing by 50 
# meaning sample about 50 times a day 

# get average daily volume
_ = df.groupby(pd.Grouper(freq='D'))['Volume'].sum()
avg_d_vol = _.replace(0, np.NaN).mean()/np.count_nonzero(_)

#get threshold
threshold = avg_d_vol/10


In [69]:
_

Date
2023-03-02    1023206.0
2023-03-03    1317372.0
2023-03-04          0.0
2023-03-05          0.0
2023-03-06    1557423.0
2023-03-07    1274807.0
2023-03-08     996490.0
2023-03-09    1956312.0
2023-03-10    1351847.0
Freq: D, Name: Volume, dtype: float64

In [65]:
#create bars    
dollar_bars = bars.get_dollar_bars(df_dict, threshold) 

#create dataframe
df_dollarbar = pd.DataFrame(dollar_bars)

#view first ten entries
df_dollarbar[80:100]

,timestamp,open,high,low,close
80,2023-03-02 11:02:00,43.599998,43.599998,43.575001,43.599998
81,2023-03-02 11:03:00,43.599998,43.630001,43.560001,43.599998
82,2023-03-02 11:04:00,43.630001,43.630001,43.593800,43.593800
83,2023-03-02 11:05:00,43.599998,43.634998,43.599998,43.634998
84,2023-03-02 11:06:00,43.549999,43.580002,43.549999,43.580002
85,2023-03-02 11:07:00,43.599998,43.599998,43.549999,43.549999
86,2023-03-02 11:09:00,43.615002,43.639999,43.615002,43.639999
87,2023-03-02 11:11:00,43.660000,43.660000,43.605000,43.650002
88,2023-03-02 11:13:00,43.650002,43.660000,43.630001,43.660000
89,2023-03-02 11:14:00,43.575001,43.639999,43.575001,43.630001


3. Run cusum filter to get std dev of daily returns at equal threshold

In [ ]:
#to do

4. Add vertical barrier

In [ ]:
#to do

5. Apply Triple Barrier method where ptSl = [2,1]  (2 to 1 profit ratio)

In [ ]:
#to do

6. Generate label using getBins

In [ ]:
#to do

7. Drop underpopulated labels (snippet 3.8)

In [ ]:
#to do 

8. Adjust getBins to 0 if vertical barrier is touched first

In [ ]:
#to do

9. Apply strategy here to suggest the side, not the bet
    
    9.1 Trendlabeling

    9.2 Mean Reverting BB 

In [ ]:
#to do

10. Train model to decide trade or not trade since step 9 already decided a side (up or down)

    10.1 Random Forest

    10.2 SVC

In [ ]:
#to do

11. Compare model with or without Secondary Model

In [ ]:
# to do